In [ ]:
import os
import pickle
import time
from parser.parser import pcapsToCSVs

import torch
from sklearn.model_selection import StratifiedKFold
from torch import nn

from gnn.graph_transformer import GraphTransformer
from graph_dataset.display_graph import displayGraph
from graph_dataset.graph_dataset import (
    loadGraphDataset,
    oversampleInfrequentClasses,
    randomizeGraphOrder,
)
from preprocessing.preprocessor import loadCSVsAndCreateGraphs
from train_evaluate.train_evaluate import evaluate, train

In [ ]:
dataset_path = "../../Datasets/TII-SSRC-23 Dataset/pcap/"
pcapsToCSVs(
    dataset_path,
    [
        "benign/audio/audio.pcap",
        "benign/background/background.pcap",
        "benign/text/text.pcap",
        "benign/video/http.pcap",
        "benign/video/rtp.pcap",
        "benign/video/udp.pcap",
    ],
    "../../Datasets/TII-SSRC-23 Dataset/My Preprocessing/CSVs/Benign",
    5000000,
    {"Benign": {}},
    0,
)

pcapsToCSVs(
    dataset_path,
    [
        "malicious/bruteforce/bruteforce_dns.pcap",
        "malicious/bruteforce/bruteforce_ftp.pcap",
        "malicious/bruteforce/bruteforce_http.pcap",
        "malicious/bruteforce/bruteforce_ssh.pcap",
        "malicious/bruteforce/bruteforce_telnet.pcap",
    ],
    "../../Datasets/TII-SSRC-23 Dataset/My Preprocessing/CSVs/Malicious/Bruteforce",
    5000000,
    {"Bruteforce": {}},
    0,
)
pcapsToCSVs(
    dataset_path,
    [
        "malicious/dos/ack_tcp_dos.pcap",
        "malicious/dos/cwr_tcp_dos.pcap",
        "malicious/dos/ecn_tcp_dos.pcap",
        "malicious/dos/http_dos.pcap",
        "malicious/dos/icmp_dos.pcap",
        "malicious/dos/mac_dos.pcap",
        "malicious/dos/psh_tcp_dos.pcap",
        "malicious/dos/rst_tcp_dos.pcap",
        "malicious/dos/syn_tcp_dos.pcap",
        "malicious/dos/udp_dos.pcap",
        "malicious/dos/urg_tcp_dos.pcap",
        "malicious/mirai-botnet/mirai_ddos_ack.pcap",
        "malicious/mirai-botnet/mirai_ddos_dns.pcap",
        "malicious/mirai-botnet/mirai_ddos_greeth.pcap",
        "malicious/mirai-botnet/mirai_ddos_greip.pcap",
        "malicious/mirai-botnet/mirai_ddos_http.pcap",
        "malicious/mirai-botnet/mirai_ddos_syn.pcap",
        "malicious/mirai-botnet/mirai_ddos_udp_udpplain.pcap",
    ],  # "Attack traffic/DDoS ICMP Flood Attacks.pcap" is not included because I only look at UDP and TCP packets
    "../../Datasets/TII-SSRC-23 Dataset/My Preprocessing/CSVs/Malicious/DOS",
    5000000,
    {"DOS": {}},
    0,
)
pcapsToCSVs(
    dataset_path,
    ["malicious/information-gathering/information_gathering.pcap"],
    "../../Datasets/TII-SSRC-23 Dataset/My Preprocessing/CSVs/Malicious/Information Gathering",
    5000000,
    {"Information Gathering": {}},
    0,
)

In [ ]:
num_nodes = 20
csvs_paths = [
    "../../Datasets/TII-SSRC-23 Dataset/My Preprocessing/CSVs/Benign/",
    "../../Datasets/TII-SSRC-23 Dataset/My Preprocessing/CSVs/Malicious/Bruteforce",
    "../../Datasets/TII-SSRC-23 Dataset/My Preprocessing/CSVs/Malicious/DOS",
    "../../Datasets/TII-SSRC-23 Dataset/My Preprocessing/CSVs/Malicious/Information Gathering",
]
graphs_path = (
    f"../../Datasets/TII-SSRC-23 Dataset/My Preprocessing/Graphs/Size {num_nodes}/"
)
loadCSVsAndCreateGraphs(csvs_paths, graphs_path, num_nodes, 10000, "Generalized", "all")

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
number_nodes = 20
epochs = 10
batch_size = 512
number_eigenvectors = 40 * 2
embedding_size = 80 * 2

In [ ]:
one_hot_mapping = {
    "Benign": 1,
    "Bruteforce": 2,
    "DOS": 2,
    "Information Gathering": 2,
}
true_labels = {
    "Benign": 1,
    "Malicious": 2,
}
graphs_path = (
    f"../../Datasets/TII-SSRC-23 Dataset/My Preprocessing/Graphs/Size {number_nodes}/"
)
(graphs, labels) = loadGraphDataset(graphs_path, one_hot_mapping, 10)
# displayGraph(graphs_train[0])

n_splits = 10
kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=101)
current_split = 1

attack_detection_best_model_params_path = os.path.join(
    "../../Checkpoints/",
    f"tii-ssrc-23-binary-cv-{number_nodes}-{number_eigenvectors}-{embedding_size}.pt",
)

In [ ]:
# Train attack detection model
print("Training attack detection model")

for train_idx, test_idx in kf.split(graphs, torch.argmax(labels, dim=1)):
    evaluation_mode = {
        "mode": "cv",
        "fold": current_split,
        "name": "tii-ssrc-23-binary-cv",
    }

    attack_detection_model = GraphTransformer(
        number_nodes=number_nodes,
        node_features_size=4,
        number_eigenvectors=number_eigenvectors,
        embedding_size=embedding_size,
        feedforward_scaling=20 * 2,
        num_heads=10 * 2,
        num_layers=4,
        dropout=0.5,
        num_classes=2,
        device=device,
    ).to(device)

    train_idx_list = train_idx.tolist()
    test_idx_list = test_idx.tolist()
    graphs_train = [graphs[i] for i in train_idx_list]
    graphs_test = [graphs[i] for i in test_idx_list]
    labels_train = labels[train_idx]
    labels_test = labels[test_idx]

    graphs_train, labels_train = oversampleInfrequentClasses(graphs_train, labels_train)
    graphs_train = [g.to(device) for g in graphs_train]
    graphs_test = [g.to(device) for g in graphs_test]
    labels_train = labels_train.to(device)  # type: ignore
    labels_test = labels_test.to(device)  # type: ignore

    loss_function = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(attack_detection_model.parameters(), lr=0.001)
    best_train_loss = float("inf")
    train_loss = float("inf")

    for epoch_num in range(1, epochs + 1):
        graphs_train, labels_train = randomizeGraphOrder(graphs_train, labels_train)
        epoch_start_time = time.time()
        # print(torch.cat((labels_train[:, :1], torch.flip(labels_train[:, :1], [1])), dim=1))
        train_loss = train(
            attack_detection_model,
            loss_function,
            optimizer,
            graphs_train,
            labels_train,
            batch_size,
            epoch_num,
            device,
            evaluation_mode,
        )
        elapsed = time.time() - epoch_start_time
        print("-" * 89)
        print(
            f"| end of epoch {epoch_num:3d} | epoch last loss {train_loss} | time: {elapsed:5.2f}s"
        )
        print("-" * 89)

        if train_loss < best_train_loss:
            print("=" * 89)
            print("| Saving new best checkpoint")
            print("=" * 89)
            best_train_loss = train_loss
            torch.save(
                attack_detection_model.state_dict(),
                attack_detection_best_model_params_path[:-3]
                + f"-fold-{current_split}.pt",
            )
    print("=" * 89)
    print("| Saving final checkpoint")
    print("=" * 89)
    torch.save(
        attack_detection_model.state_dict(),
        attack_detection_best_model_params_path[:-3]
        + f"-fold-{current_split}-final.pt",
    )
    results = evaluate(
        attack_detection_model,
        attack_detection_best_model_params_path[:-3] + f"-fold-{current_split}.pt",
        loss_function,
        graphs_test,
        labels_test,
        list(true_labels.keys()),
        batch_size,
        device,
        evaluation_mode,
    )
    with open(
        f"Results/Pickle/tii-ssrc-23-binary-cv-fold-{current_split}-results.pkl",
        "wb",
    ) as file:
        pickle.dump(results, file)
    current_split += 1

# Show results
results = []
for i in range(0, n_splits):
    with open(
        f"Results/Pickle/tii-ssrc-23-binary-cv-fold-{i+1}-results.pkl", "rb"
    ) as file:
        results.append(pickle.load(file))
    print(f"Fold {i+1}")
    print("=" * 89)
    print(
        f"| accuracy: {results[i]['accuracy']} "
        f"| macro precision: {results[i]['precision']}\n"
        f"| macro recall: {results[i]['recall']} "
        f"| macro f1-score: {results[i]['f1_score']}"
    )
    print("=" * 89)
    print("Classification Report")
    print(results[i]["cr"])
    print("=" * 89)


print("=" * 89)
print("Mean Fold Metrics")
print("=" * 89)
print(
    f"| mean accuracy: {sum(results[i]['accuracy'] for i in range(0,n_splits)) / n_splits} "
    f"| mean macro precision: {sum(results[i]['precision'] for i in range(0,n_splits)) / n_splits}\n"
    f"| mean macro recall: {sum(results[i]['recall'] for i in range(0, n_splits)) / n_splits} "
    f"| mean macro f1-score: {sum(results[i]['f1_score'] for i in range(0, n_splits)) / n_splits}"
)
print("=" * 89)